In [1]:
#Importing Libraries 
import pandas as pd
import statistics

In [2]:
df = pd.read_csv("Final_CleanedData.csv") #Reading in Dataframe 
data = df[['id' , 'domain_id', 'batch_id' , 'ean_13', 'seller_name' ,  'found' , 'type' , 'sold_by_third_party' , 
           'fulfilled_by_third_party', 'has_buy_button' , 'started_at' , 'primary_sale_type']] #Creating a subset of data with specific fields 
data.head()

/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,domain_id,batch_id,ean_13,seller_name,found,type,sold_by_third_party,fulfilled_by_third_party,has_buy_button,started_at,primary_sale_type
0,124273619,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781402738159,SuperBookDeals---,t,Hardcover,t,t,t,18/11/2019 20:12:05.936306,For Sale
1,124280803,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454919735,SuperBookDeals---,t,Board book,t,t,t,18/11/2019 20:12:05.936306,For Sale
2,124281694,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454922377,SuperBookDeals---,t,Hardcover,t,t,t,18/11/2019 20:12:05.936306,For Sale
3,124284134,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454929284,SuperBookDeals---,t,Board book,t,t,t,18/11/2019 20:12:05.936306,For Sale
4,124284363,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454929666,SuperBookDeals---,t,Board book,t,t,t,18/11/2019 20:12:05.936306,For Sale


In [3]:
#Date Column
data['started_at'] = pd.to_datetime(data['started_at'] , dayfirst = True) #Converting column to datetime form 
data['Year'] = data['started_at'].dt.year #Creating year column
data['Month'] = data['started_at'].dt.month #Creating Month column
data['Day'] = data['started_at'].dt.day #Creating Day Column

/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [4]:
#Creating a dataframe for each month
Months = list(data['Month'].unique()) #Creating a list of the months 
Months_df = [] 
for month in Months: 
    Months_df.append(data.loc[data['Month'] == month])  

In [7]:
stats_data_dfs = [] 
for mdf in Months_df: 
    
    Days = list(mdf['Day'].unique()) #Create a list of the days 
    ld = len(Days) #Length of Days 
    
    
    saletype = list(mdf['primary_sale_type'].unique()) #Creating a list of sale types
    
    #Creating a dataframe for each primary sale type
    saletype_dfs = [] 
    for t in saletype: 
        saletype_dfs.append(mdf.loc[mdf['primary_sale_type'] == t])
        
    
    new_data_dfs = [] 
    for d1 in saletype_dfs: #Looping through dataframe by sale type
        
        products = list(d1['ean_13'].unique()) #creating a list of products
        

        avg_buywins = [] 
        avg_percent = []
        for i in products: 
            buycount = [] 
            buypct = []
            d = d1.loc[d1['ean_13'] == i]
            buycount.append(d['has_buy_button'].count())

            for b in buycount: 
                buypct.append((b/ld) * 100)



            avg_buywins.append(statistics.mean(buycount))
            avg_percent.append(statistics.mean(buypct))
        
        
            
        #Creating dataframe with final numbers  
        df_prod = pd.DataFrame({'Product': products, 
                                'Avg Buy Wins': avg_buywins, 
                                'Avg Buy Wins Percentage': avg_percent})

        new_data_dfs.append(df_prod)
    
    for tp , d2 in zip(saletype, new_data_dfs):
        d2['Primary_Sale_Type'] = tp
        
    mdata = pd.concat(new_data_dfs)
    
    stats_data_dfs.append(mdata)

In [8]:
for m , d4 in zip(Months,  stats_data_dfs): 
    d4['Month'] = m

In [9]:
finaldata = pd.concat(stats_data_dfs) #Merging Data together 
finaldata.head()

,Product,Avg Buy Wins,Avg Buy Wins Percentage,Primary_Sale_Type,Month
0,9.781403e+12,4.0,13.333333,For Sale,11
1,9.781455e+12,27.0,90.000000,For Sale,11
2,9.781455e+12,21.0,70.000000,For Sale,11
3,9.781455e+12,15.0,50.000000,For Sale,11
4,9.781455e+12,9.0,30.000000,For Sale,11


In [ ]:
typesale = list(finaldata['Primary_Sale_Type'].unique()) #Creating a list of sale type 
dfs_saletype = [] 
for t in typesale:
    dfs_saletype.append(finaldata.loc[finaldata['Primary_Sale_Type'] == t]) #Creating a dataframe for each sale type

In [11]:
n_datadfs = [] 
for dt in dfs_saletype: 
    
    products1 = list(dt['Product'].unique())
    
    df2_prod = []
    for s in products1:
        
        d = dt.loc[dt['Product'] == s]
        d1 = d[['Product', 'Avg Buy Wins' , 'Avg Buy Wins Percentage' ]]
        months = list(d['Month'])
        saletype = list(d['Primary_Sale_Type'])
        perc = list(d['Avg Buy Wins Percentage'])
        wins = list(d['Avg Buy Wins'])


        diff1 = d1.set_index('Product').diff()

        diff2 = diff1.reset_index()
        diff2['Month'] = months
        diff2['Primary_Sale_Type'] = saletype
        diff2['AvgWinsPercent'] = perc
        diff2['AvgWins'] = wins
        df2_prod.append(diff2)
        
        
        
    dfinal = pd.concat(df2_prod)
    n_datadfs.append(dfinal)

In [12]:
final_data2 = pd.concat(n_datadfs)
final_data2.head()

,Product,Avg Buy Wins,Avg Buy Wins Percentage,Month,Primary_Sale_Type,AvgWinsPercent,AvgWins
0,9.781403e+12,NaN,NaN,11,For Sale,13.333333,4.0
1,9.781403e+12,-3.0,-10.000000,12,For Sale,3.333333,1.0
2,9.781403e+12,19.0,61.182796,1,For Sale,64.516129,20.0
3,9.781403e+12,-12.0,-36.929922,2,For Sale,27.586207,8.0
0,9.781455e+12,NaN,NaN,11,For Sale,90.000000,27.0


In [13]:
final_data2.columns = ['Product','Change in Avg Buy Wins', 'Change in Avg Buy Wins Percentage', 'Month', 'Primary_Sale_Type', 'AvgWinsPercent', 'AvgWins']



In [14]:
finalData3 = final_data2.fillna(value=0)


category = [] 
for i in finalData3['Change in Avg Buy Wins Percentage']: 
    if i < 0 : 
        category.append("Percentage Descrease")
    elif i == 0: 
        category.append("No Change")
    else: 
        category.append("Percenatge Increase")
        
        
finalData3['Change Percentage Category'] = category
finalData3.head()

,Product,Change in Avg Buy Wins,Change in Avg Buy Wins Percentage,Month,Primary_Sale_Type,AvgWinsPercent,AvgWins,Change Percentage Category
0,9.781403e+12,0.0,0.000000,11,For Sale,13.333333,4.0,No Change
1,9.781403e+12,-3.0,-10.000000,12,For Sale,3.333333,1.0,Percentage Descrease
2,9.781403e+12,19.0,61.182796,1,For Sale,64.516129,20.0,Percenatge Increase
3,9.781403e+12,-12.0,-36.929922,2,For Sale,27.586207,8.0,Percentage Descrease
0,9.781455e+12,0.0,0.000000,11,For Sale,90.000000,27.0,No Change


In [ ]:
finalData3.to_csv('ThirdPartyProducts_Difference.csv' , index = False) #Save to csv 
finalData3.to_excel("Products_Difference.xlsx") #Save to excel 